In [121]:
import ipaddress
import logging

# Sample connection data
connections = [
    {'san-q5230-05': 'et-0/0/8', 'san-q5230-06': 'et-0/0/8'},
    {'san-q5230-06': 'et-0/0/1', 'san-q5240-18': 'et-0/0/0:1'},
    {'san-q5230-06': 'et-0/0/8', 'san-q5230-05': 'et-0/0/8'},
    {'san-q5240-18': 'et-0/0/0:1', 'san-q5230-06': 'et-0/0/1'}
]

# Sample user input for IP type selection
use_ipv4 = True  # Set based on user selection for IPv4
use_ipv6 = True  # Set based on user selection for IPv6

ip_assignments = {}
subnet_counter = 1
configured_interfaces = {}
configured_groups = set()  # To keep track of which groups have been configured
skip_interfaces = {'re0:mgmt-0', 'em0', 'fxp0'}
skip_device_patterns = ['mgmt', 'management', 'hypercloud']
local_as_mapping = {
    'san-q5230-05': 65000,
    'san-q5230-06': 65001,
    'san-q5240-18': 65002
}

# Functions
def get_ip(subnet_counter, host_id, ipv6=False):
    if ipv6:
        # Generate IPv6 address using a simple subnet structure for testing
        return f"fd00:{subnet_counter}::{host_id}"
    else:
        # Generate IPv4 address
        return f"192.168.{subnet_counter}.{host_id}"

def get_subnet(ip_address, ipv6=False):
    if ipv6:
        return ipaddress.ip_network(ip_address + '/64', strict=False)
    else:
        return ipaddress.ip_network(ip_address + '/30', strict=False)

def generate_bgp_group_config(group_name, ipv6=False):
    family = 'inet6 unicast' if ipv6 else 'inet unicast'
    return [
        f"set protocols bgp group {group_name} family {family}"
    ]

def generate_bgp_neighbor_config(local_ip, neighbor_ip, local_as, remote_as, group_name):
    return [
        "## BGP Neighbor Config ##",
        f"set protocols bgp group {group_name} neighbor {neighbor_ip} peer-as {remote_as}",
        f"set protocols bgp group {group_name} neighbor {neighbor_ip} local-address {local_ip}",
        f"set protocols bgp group {group_name} local-as {local_as}"
    ]

# Process connections
for connection in connections:
    subnet = subnet_counter
    subnet_counter += 1
    host_id = 1  # Reset host_id for each new connection
    neighbor_ip_mapping = {}

    # First loop: Assign IP addresses to devices and populate neighbor_ip_mapping
    if isinstance(connection, dict):
        for device, interface in connection.items():
            if any(pattern in device.lower() for pattern in skip_device_patterns) or interface in skip_interfaces:
                continue
            if device not in ip_assignments:
                ip_assignments[device] = []
            if device not in configured_interfaces:
                configured_interfaces[device] = set()

            if interface not in configured_interfaces[device]:
                # Assign unique IP addresses based on user's choice (IPv4, IPv6, or both)
                if use_ipv4:
                    ipv4_address = get_ip(subnet, host_id, ipv6=False)
                    ip_assignments[device].append((interface, ipv4_address))
                    configured_interfaces[device].add(interface)
                    neighbor_ip_mapping[f"{device}-{interface}-ipv4"] = ipv4_address  # Store IPv4 address

                if use_ipv6:
                    ipv6_address = get_ip(subnet, host_id, ipv6=True)
                    ip_assignments[device].append((interface, ipv6_address))
                    configured_interfaces[device].add(interface)
                    neighbor_ip_mapping[f"{device}-{interface}-ipv6"] = ipv6_address  # Store IPv6 address

                # Increment host_id for next device
                host_id += 1

        # Debugging: Print neighbor_ip_mapping only if it's not empty
        if neighbor_ip_mapping:
            print(f"Neighbor IP mapping after first loop: {neighbor_ip_mapping}")

    # Second loop: Use the populated neighbor_ip_mapping to create BGP configurations
    for device, interface in connection.items():
        for ip_version in ["ipv4", "ipv6"]:
            if ip_version == "ipv4" and not use_ipv4:
                continue
            if ip_version == "ipv6" and not use_ipv6:
                continue

            if f"{device}-{interface}-{ip_version}" not in neighbor_ip_mapping:
                # Skip if the interface was not assigned an IP address in the first loop
                continue

            local_subnet = get_subnet(neighbor_ip_mapping[f"{device}-{interface}-{ip_version}"], ipv6=(ip_version == "ipv6"))

            for remote_device, remote_interface in connection.items():
                if remote_device != device and f"{remote_device}-{remote_interface}-{ip_version}" in neighbor_ip_mapping:
                    remote_subnet = get_subnet(neighbor_ip_mapping[f"{remote_device}-{remote_interface}-{ip_version}"], ipv6=(ip_version == "ipv6"))
                    if local_subnet == remote_subnet:
                        local_as = local_as_mapping.get(device)
                        remote_as = local_as_mapping.get(remote_device)

                        if local_as is not None and remote_as is not None:
                            neighbor_ip = neighbor_ip_mapping[f"{remote_device}-{remote_interface}-{ip_version}"]  # Neighbor's IP
                            local_ip = neighbor_ip_mapping[f"{device}-{interface}-{ip_version}"]  # Local interface IP

                            # Select BGP group name based on IP version
                            group_name = "underlay_v4" if ip_version == "ipv4" else "underlay_v6"

                            # Configure BGP group if not already done
                            if group_name not in configured_groups:
                                group_commands = generate_bgp_group_config(group_name, ipv6=(ip_version == "ipv6"))
                                print(f"BGP group configuration for {group_name}: {group_commands}")
                                configured_groups.add(group_name)

                            # Generate BGP neighbor configuration for the device
                            bgp_commands = generate_bgp_neighbor_config(local_ip, neighbor_ip, local_as, remote_as, group_name)
                            print(f"BGP neighbor configuration for {device}: {bgp_commands}")


Neighbor IP mapping after first loop: {'san-q5230-05-et-0/0/8-ipv4': '192.168.1.1', 'san-q5230-05-et-0/0/8-ipv6': 'fd00:1::1', 'san-q5230-06-et-0/0/8-ipv4': '192.168.1.2', 'san-q5230-06-et-0/0/8-ipv6': 'fd00:1::2'}
BGP group configuration for underlay_v4: ['set protocols bgp group underlay_v4 family inet unicast']
BGP neighbor configuration for san-q5230-05: ['## BGP Neighbor Config ##', 'set protocols bgp group underlay_v4 neighbor 192.168.1.2 peer-as 65001', 'set protocols bgp group underlay_v4 neighbor 192.168.1.2 local-address 192.168.1.1', 'set protocols bgp group underlay_v4 local-as 65000']
BGP group configuration for underlay_v6: ['set protocols bgp group underlay_v6 family inet6 unicast']
BGP neighbor configuration for san-q5230-05: ['## BGP Neighbor Config ##', 'set protocols bgp group underlay_v6 neighbor fd00:1::2 peer-as 65001', 'set protocols bgp group underlay_v6 neighbor fd00:1::2 local-address fd00:1::1', 'set protocols bgp group underlay_v6 local-as 65000']
BGP neighb

In [162]:

## building connections##
def build_connections_from_sample_data():
    # Sample data
    data = [
        {"device1": "san-q5230-05", "interface1": "et-0/0/8", "device2": "san-q5230-06", "interface2": "et-0/0/8"},
        {"device1": "san-q5230-05", "interface1": "et-0/0/9", "device2": "san-q5230-06", "interface2": "et-0/0/9"}
    ]
    
    connections = []
    seen = set()
    duplicates = []
    duplicate_device_interfaces = []

    # Iterate over the sample data and build the connections
    for row in data:
        # Create a tuple to represent the device-interface pairs for both sides of the connection
        connection_tuple_1 = (row['device1'], row['interface1'])
        connection_tuple_2 = (row['device2'], row['interface2'])
        
        # Check for duplicates based on either side of the connection
        if connection_tuple_1 in seen or connection_tuple_2 in seen:
            duplicates.append({row['device1']: row['interface1'], row['device2']: row['interface2']})
            if connection_tuple_1 in seen:
                duplicate_device_interfaces.append(f"Duplicate device/interface found: {row['device1']} using {row['interface1']}")
            if connection_tuple_2 in seen:
                duplicate_device_interfaces.append(f"Duplicate device/interface found: {row['device2']} using {row['interface2']}")
        else:
            # Add both sides of the connection to the seen set
            seen.add(connection_tuple_1)
            seen.add(connection_tuple_2)
            # Add the unique connection
            connections.append({row['device1']: row['interface1'], row['device2']: row['interface2']})
    
    if duplicates:
        print("Duplicate data found (same device/interface used multiple times):")
        for duplicate in duplicates:
            print(duplicate)
        print("\nDuplicate device/interface details:")
        for detail in duplicate_device_interfaces:
            print(detail)
    
    return connections

# Example usage
connections = build_connections_from_sample_data()
print(connections)

delete_underlay_group=True
use_ipv4 = True
use_ipv6 = False
#print(f"delete_underlay_group: {delete_underlay_group}")
commands = defaultdict(list)
local_as_mapping = {}
remote_as_mapping = {}
ip_assignments = {}
neighbors_dict = defaultdict(list)
as_counter = 65000
success_hosts = []
failed_hosts = set()

def is_valid_port_desc(port_desc):
    # Valid interface patterns (modify as per your environment)
    interface_pattern = r"^(et|ge|xe|em|re|fxp)\-[0-9]+\/[0-9]+\/[0-9]+(:[0-9]+)?$"
    return bool(re.match(interface_pattern, port_desc))

for a,b in x.items():
    for c in b:
        print(c)
for connection in connections:
    print(f"connections: {connection}")
    for device, interface in connection.items():
        print(f"device: {device}, Interface: {interface}")
    

[{'san-q5230-05': 'et-0/0/8', 'san-q5230-06': 'et-0/0/8'}, {'san-q5230-05': 'et-0/0/9', 'san-q5230-06': 'et-0/0/9'}]
CSV Connection: {'san-q5230-05': 'et-0/0/8', 'san-q5230-06': 'et-0/0/8'}
Neighbor IP mapping after first loop: {'san-q5230-05-et-0/0/8-ipv4': '192.168.1.1', 'san-q5230-06-et-0/0/8-ipv4': '192.168.1.2'}
**************************************************
CSV Connection: {'san-q5230-05': 'et-0/0/9', 'san-q5230-06': 'et-0/0/9'}
Neighbor IP mapping after first loop: {'san-q5230-05-et-0/0/9-ipv4': '192.168.2.1', 'san-q5230-06-et-0/0/9-ipv4': '192.168.2.2'}
**************************************************


'for a,b in x.items():\n    for c in b:\n        print(c)\nfor connection in connections:\n    print(f"connections: {connection}")\n    for device, interface in connection.items():\n        print(f"device: {device}, Interface: {interface}")'

In [184]:
connection=[{'san-q5230-05': 'et-0/0/8', 'san-q5230-06': 'et-0/0/8'}, {'san-q5230-05': 'et-0/0/9', 'san-q5230-06': 'et-0/0/9'}]
neighbor_ip_mapping= {'san-q5230-05-et-0/0/9-ipv4': '192.168.2.1', 'san-q5230-06-et-0/0/9-ipv4': '192.168.2.2'}

def generate_config(commands, connections, local_as_mapping, delete_underlay_group, use_ipv4, use_ipv6, ip_assignments=None):
    ip_assignments = ip_assignments or {}
    subnet_counter = 1
    configured_groups = set()  # Track which BGP groups have been configured
    skip_interfaces = {'re0:mgmt-0', 'em0', 'fxp0'}
    skip_device_patterns = ['mgmt', 'management', 'hypercloud']
    configured_interfaces = {}

    devices = ["san-q5230-05","san-q5230-06"]
    as_counter = 65000
    if not devices:
        logging.info(f"No devices found for user {current_user.id}")
        flash('No devices found for the current user.', 'error')
        return redirect(url_for('index'))

    for device in devices:
        if device not in local_as_mapping:
            local_as_mapping[device] = as_counter
            as_counter += 1
                    
    def get_ip(subnet_counter, host_id, ipv6=False):
        if ipv6:
            return f"fd00:{subnet_counter}::{host_id}"
        else:
            return f"192.168.{subnet_counter}.{host_id}"

    def get_subnet(ip_address, ipv6=False):
        if ipv6:
            return ipaddress.ip_network(ip_address + '/64', strict=False)
        else:
            return ipaddress.ip_network(ip_address + '/30', strict=False)

    def generate_bgp_group_config(group_name, use_ipv4=False, use_ipv6=False):
        commands = []
        # Add family inet unicast for IPv4 group
        if use_ipv4 and group_name == "underlay_v4":
            commands.append(f"set protocols bgp group {group_name} family inet unicast")

        # Add family inet6 unicast for IPv6 group
        if use_ipv6 and group_name == "underlay_v6":
            commands.append(f"set protocols bgp group {group_name} family inet6 unicast")

        return commands

    def generate_interface_group_config(interface, ipv4_address=None, ipv6_address=None):
        config_commands = [f"delete interfaces {interface}"]
        #logging.info(f"Configuring interface {interface} with addresses: IPv4: {ipv4_address}, IPv6: {ipv6_address}")

        # For IPv4, use /30 prefix length
        if ipv4_address:
            config_commands.append(f"set interfaces {interface} unit 0 family inet address {ipv4_address}/30")

        # For IPv6, use /64 prefix length
        if ipv6_address:
            config_commands.append(f"set interfaces {interface} unit 0 family inet6 address {ipv6_address}/64")

        return config_commands

    def generate_bgp_neighbor_config(local_ip, neighbor_ip, local_as, remote_as, group_name):
        return [
            "## BGP Neighbor Config ##",
            f"set protocols bgp group {group_name} neighbor {neighbor_ip} peer-as {remote_as}",
            f"set protocols bgp group {group_name} neighbor {neighbor_ip} local-address {local_ip}",
            f"set protocols bgp group {group_name} local-as {local_as}"
        ]

    def remove_duplicates(commands_list):
        seen = set()
        result = []
        for command in commands_list:
            if command not in seen:
                seen.add(command)
                result.append(command)
        return result

    #logging.info(f"connections:generate_config-utile.py {connections}")

    # Process connections
    for connection in connections:
        print(f"CSV Connection: {connection}")
        subnet = subnet_counter
        subnet_counter += 1
        host_id = 1  # Reset host_id for each new connection
        neighbor_ip_mapping = {}

        # First loop: Assign IP addresses to devices and populate neighbor_ip_mapping
        if isinstance(connection, dict):
            for device, interface in connection.items():
                #print(f"device: {device}, Interface: {interface}")
                if any(pattern in device.lower() for pattern in skip_device_patterns) or interface in skip_interfaces:
                    continue
                if device not in ip_assignments:
                    ip_assignments[device] = {}
                if device not in configured_interfaces:
                    configured_interfaces[device] = set()

                if interface not in configured_interfaces[device]:
                    # Assign unique IP addresses based on user's choice (IPv4, IPv6, or both)
                   
                    ipv4_address, ipv6_address = None, None
                    if use_ipv4:
                        ipv4_address = get_ip(subnet, host_id, ipv6=False)
                        ip_assignments[device][interface] = {"ipv4": ipv4_address}
                        neighbor_ip_mapping[f"{device}-{interface}-ipv4"] = ipv4_address

                    if use_ipv6:
                        ipv6_address = get_ip(subnet, host_id, ipv6=True)
                        ip_assignments[device][interface]["ipv6"] = ipv6_address
                        neighbor_ip_mapping[f"{device}-{interface}-ipv6"] = ipv6_address

                    configured_interfaces[device].add(interface)

                    # Increment host_id for next device
                    host_id += 1

            # Debugging: Print neighbor_ip_mapping only if it's not empty
            if neighbor_ip_mapping:
                print(f"Neighbor IP mapping after first loop: {neighbor_ip_mapping}")
                #logging.info(f"Neighbor IP mapping after first loop: {neighbor_ip_mapping}")
        #logging.info(f"Neighbor IP Mapping: {neighbor_ip_mapping}")
        # Second loop: Use the populated neighbor_ip_mapping to create BGP configurations
            for ip_version in ["ipv4", "ipv6"]:
                if ip_version == "ipv4" and not use_ipv4:
                    continue
                if ip_version == "ipv6" and not use_ipv6:
                    continue

                if f"{device}-{interface}-{ip_version}" not in neighbor_ip_mapping:
                    # Skip if the interface was not assigned an IP address in the first loop
                    continue

                local_subnet = get_subnet(neighbor_ip_mapping[f"{device}-{interface}-{ip_version}"], ipv6=(ip_version == "ipv6"))

                for remote_device, remote_interface in connection.items():
                    
                    if remote_device != device and f"{remote_device}-{remote_interface}-{ip_version}" in neighbor_ip_mapping:
                        remote_subnet = get_subnet(neighbor_ip_mapping[f"{remote_device}-{remote_interface}-{ip_version}"], ipv6=(ip_version == "ipv6"))
                        print("**************************************************")
                        if local_subnet == remote_subnet:
                            local_as = local_as_mapping.get(device)
                            remote_as = local_as_mapping.get(remote_device)
                            
                            if local_as is not None and remote_as is not None:
                                print("--------------------------")
                                neighbor_ip = neighbor_ip_mapping[f"{remote_device}-{remote_interface}-{ip_version}"]
                                
                                local_ip = neighbor_ip_mapping[f"{device}-{interface}-{ip_version}"]

                                # Select BGP group name based on IP version
                                group_name = "underlay_v4" if ip_version == "ipv4" else "underlay_v6"

                                # Configure BGP group if not already done
                                if group_name not in configured_groups:
                                    group_commands = generate_bgp_group_config(group_name,
                                                                               use_ipv4=(ip_version == "ipv4"),
                                                                               use_ipv6=(ip_version == "ipv6"))
                                    commands[device].extend(group_commands)
                                    configured_groups.add(group_name)

                                # Generate BGP neighbor configuration for the device
                                bgp_commands = generate_bgp_neighbor_config(local_ip, neighbor_ip, local_as, remote_as, group_name)
                                print(bgp_commands)
                                commands[device].extend(bgp_commands)

    # Add common and interface configurations
    for device, interfaces in ip_assignments.items():
        if any(pattern in device.lower() for pattern in skip_device_patterns):
            continue
        if device not in commands:
            commands[device] = []
        if delete_underlay_group:
            if use_ipv4:
                commands[device].insert(0, f"delete protocols bgp group underlay_v4")
            if use_ipv6:
                commands[device].insert(1, f"delete protocols bgp group underlay_v6")

        # Add common config and interface configs
        commands[device].extend(generate_common_config(use_ipv4,use_ipv6))
        for interface, ip_data in interfaces.items():
            if use_ipv4:
                ipv4_address = ip_data.get("ipv4")
            if use_ipv6:
                ipv6_address = ip_data.get("ipv6")
            logging.info(f"Generating config for interface {interface}: IPv4: {ipv4_address}, IPv6: {ipv6_address}")
            commands[device].extend(generate_interface_group_config(interface, ipv4_address, ipv6_address))

        # Remove duplicate commands before logging or further usage
        commands[device] = remove_duplicates(commands[device])
    return commands

use_ipv6=True
use_ipv4=False
generate_config(commands, connections, local_as_mapping, delete_underlay_group,use_ipv4, use_ipv6, ip_assignments)


CSV Connection: {'san-q5230-05': 'et-0/0/8', 'san-q5230-06': 'et-0/0/8'}


KeyError: 'et-0/0/8'

In [197]:
def generate_config(commands, connections, local_as_mapping, delete_underlay_group, use_ipv4, use_ipv6, ip_assignments=None):
    ip_assignments = ip_assignments or {}
    subnet_counter = 1
    configured_groups = set()  # Track which BGP groups have been configured
    skip_interfaces = {'re0:mgmt-0', 'em0', 'fxp0'}
    skip_device_patterns = ['mgmt', 'management', 'hypercloud']
    configured_interfaces = {}

    devices = ["san-q5230-05", "san-q5230-06"]
    as_counter = 65000
    if not devices:
        logging.info(f"No devices found for user {current_user.id}")
        flash('No devices found for the current user.', 'error')
        return redirect(url_for('index'))

    for device in devices:
        if device not in local_as_mapping:
            local_as_mapping[device] = as_counter
            as_counter += 1

    def get_ip(subnet_counter, host_id, ipv6=False):
        if ipv6:
            return f"fd00:{subnet_counter}::{host_id}"
        else:
            return f"192.168.{subnet_counter}.{host_id}"

    def get_subnet(ip_address, ipv6=False):
        if ipv6:
            return ipaddress.ip_network(ip_address + '/64', strict=False)
        else:
            return ipaddress.ip_network(ip_address + '/30', strict=False)

    def generate_bgp_group_config(group_name, use_ipv4, use_ipv6):
        commands = []
        if use_ipv4 and group_name == "underlay_v4":
            commands.append(f"set protocols bgp group {group_name} family inet unicast")

        if use_ipv6 and group_name == "underlay_v6":
            commands.append(f"set protocols bgp group {group_name} family inet6 unicast")

        return commands

    def generate_interface_group_config(interface, ipv4_address=None, ipv6_address=None):
        config_commands = [f"delete interfaces {interface}"]
        if ipv4_address:
            config_commands.append(f"set interfaces {interface} unit 0 family inet address {ipv4_address}/30")
        if ipv6_address:
            config_commands.append(f"set interfaces {interface} unit 0 family inet6 address {ipv6_address}/64")
        return config_commands

    def generate_bgp_neighbor_config(local_ip, neighbor_ip, local_as, remote_as, group_name):
        return [
            "## BGP Neighbor Config ##",
            f"set protocols bgp group {group_name} neighbor {neighbor_ip} peer-as {remote_as}",
            f"set protocols bgp group {group_name} neighbor {neighbor_ip} local-address {local_ip}",
            f"set protocols bgp group {group_name} local-as {local_as}"
        ]

    def remove_duplicates(commands_list):
        seen = set()
        result = []
        for command in commands_list:
            if command not in seen:
                seen.add(command)
                result.append(command)
        return result

    # Process connections
    for connection in connections:
        subnet = subnet_counter
        subnet_counter += 1
        host_id = 1  # Reset host_id for each new connection
        neighbor_ip_mapping = {}

        if isinstance(connection, dict):
            for device, interface in connection.items():
                if any(pattern in device.lower() for pattern in skip_device_patterns) or interface in skip_interfaces:
                    continue
                if device not in ip_assignments:
                    ip_assignments[device] = {}
                if device not in configured_interfaces:
                    configured_interfaces[device] = set()

                if interface not in configured_interfaces[device]:
                    # Ensure ip_assignments[device][interface] is initialized
                    if interface not in ip_assignments[device]:
                        ip_assignments[device][interface] = {}

                    # Assign unique IP addresses based on user's choice (IPv4, IPv6, or both)
                    ipv4_address, ipv6_address = None, None
                    if use_ipv4:
                        ipv4_address = get_ip(subnet, host_id, ipv6=False)
                        ip_assignments[device][interface]["ipv4"] = ipv4_address
                        neighbor_ip_mapping[f"{device}-{interface}-ipv4"] = ipv4_address

                    if use_ipv6:
                        ipv6_address = get_ip(subnet, host_id, ipv6=True)
                        ip_assignments[device][interface]["ipv6"] = ipv6_address
                        neighbor_ip_mapping[f"{device}-{interface}-ipv6"] = ipv6_address

                    configured_interfaces[device].add(interface)

                    # Increment host_id for the next device
                    host_id += 1

            # Debugging: Print neighbor_ip_mapping only if it's not empty
            if neighbor_ip_mapping:
                print(f"Neighbor IP mapping after first loop: {neighbor_ip_mapping}")

        # Second loop: Use the populated neighbor_ip_mapping to create BGP configurations
        for ip_version in ["ipv4", "ipv6"]:
            if ip_version == "ipv4" and not use_ipv4:
                continue
            if ip_version == "ipv6" and not use_ipv6:
                continue

            if f"{device}-{interface}-{ip_version}" not in neighbor_ip_mapping:
                continue

            local_subnet = get_subnet(neighbor_ip_mapping[f"{device}-{interface}-{ip_version}"], ipv6=(ip_version == "ipv6"))

            for remote_device, remote_interface in connection.items():
                if remote_device != device and f"{remote_device}-{remote_interface}-{ip_version}" in neighbor_ip_mapping:
                    remote_subnet = get_subnet(neighbor_ip_mapping[f"{remote_device}-{remote_interface}-{ip_version}"], ipv6=(ip_version == "ipv6"))
                    if local_subnet == remote_subnet:
                        local_as = local_as_mapping.get(device)
                        remote_as = local_as_mapping.get(remote_device)

                        if local_as is not None and remote_as is not None:
                            neighbor_ip = neighbor_ip_mapping[f"{remote_device}-{remote_interface}-{ip_version}"]
                            local_ip = neighbor_ip_mapping[f"{device}-{interface}-{ip_version}"]

                            group_name = "underlay_v4" if ip_version == "ipv4" else "underlay_v6"

                            if group_name not in configured_groups:
                                group_commands = generate_bgp_group_config(group_name, use_ipv4=(ip_version == "ipv4"), use_ipv6=(ip_version == "ipv6"))
                                commands[device].extend(group_commands)
                                configured_groups.add(group_name)

                            bgp_commands = generate_bgp_neighbor_config(local_ip, neighbor_ip, local_as, remote_as, group_name)
                            commands[device].extend(bgp_commands)

    # Add common and interface configurations
    for device, interfaces in ip_assignments.items():
        if any(pattern in device.lower() for pattern in skip_device_patterns):
            continue
        if device not in commands:
            commands[device] = []
        if delete_underlay_group:
            if use_ipv4:
                commands[device].insert(0, f"delete protocols bgp group underlay_v4")
            if use_ipv6:
                commands[device].insert(1, f"delete protocols bgp group underlay_v6")

        # Add common config only if IPv4 or IPv6 is enabled
        if use_ipv4 or use_ipv6:
            commands[device].extend(generate_common_config(use_ipv4, use_ipv6))
        
        # Add interface configurations based on the flag
        for interface, ip_data in interfaces.items():
            ipv4_address = ip_data.get("ipv4") if use_ipv4 else None
            ipv6_address = ip_data.get("ipv6") if use_ipv6 else None

            if ipv4_address or ipv6_address:
                commands[device].extend(generate_interface_group_config(interface, ipv4_address, ipv6_address))

        # Remove duplicate commands
        commands[device] = remove_duplicates(commands[device])

    return commands


use_ipv6=False
use_ipv4=True
generate_config(commands, connections, local_as_mapping, delete_underlay_group,use_ipv4=False, use_ipv6=False, ip_assignments=None)


defaultdict(list,
            {'san-q5230-05': ['delete protocols bgp group underlay_v4',
              'delete protocols bgp group underlay_v6',
              'set protocols lldp interface all',
              'set policy-options policy-statement export_v4_lo0 term 1 from interface lo0',
              'set policy-options policy-statement export_v4_lo0 term 1 then accept',
              'set protocols bgp group underlay_v4 export export_v4_lo0',
              'set protocols bgp group underlay_v4 type external',
              'delete interfaces et-0/0/8',
              'set interfaces et-0/0/8 unit 0 family inet address 192.168.1.1/30',
              'delete interfaces et-0/0/9',
              'set interfaces et-0/0/9 unit 0 family inet address 192.168.2.1/30',
              'set policy-options policy-statement export_v6_lo0 term 1 from interface lo0',
              'set policy-options policy-statement export_v6_lo0 term 1 from rib inet6.0',
              'set policy-options policy-state

True
